In [1]:
import gc
import time
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
import xgboost as xgb
from xgboost import plot_importance
import matplotlib.pyplot as plt

/home/ubuntu/.pyenv/versions/3.6.4/envs/talkingdata/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [14]:
%%time
click_count_by_ip = pd.read_hdf("../data/click_count_by_ip.hdf", "click_count_by_ip")

CPU times: user 8 ms, sys: 12 ms, total: 20 ms
Wall time: 16 ms


In [2]:
%%time
click_data = pd.read_hdf("../data/click_data.hdf", "click_data")

CPU times: user 13.2 s, sys: 9.77 s, total: 23 s
Wall time: 3min 6s


In [24]:
%%time
click_data["rank_by_ip"] = click_data.groupby("ip").cumcount()

CPU times: user 35.5 s, sys: 9.47 s, total: 45 s
Wall time: 45 s


In [28]:
rank_by_ip=click_data[["rank_by_ip"]]

In [30]:
rank_by_ip.to_hdf("../data/rank_by_ip.hdf", "rank_by_ip")

In [14]:
%%time
ip_os_device = pd.read_hdf("../data/ip_os_device.hdf", "ip_os_device")

CPU times: user 14.8 s, sys: 6.65 s, total: 21.5 s
Wall time: 1min 9s


In [15]:
%%time
ip_os_device_dow = pd.read_hdf("../data/ip_os_device_dow.hdf", "ip_os_device_dow")

CPU times: user 1.71 s, sys: 1.2 s, total: 2.92 s
Wall time: 1min 17s


In [17]:
%%time
click_data = pd.concat([click_data, ip_os_device, ip_os_device_dow], axis=1)

CPU times: user 9.82 s, sys: 5.47 s, total: 15.3 s
Wall time: 15.3 s


In [37]:
%%time
click_data["rank_by_ip_os_device"] = click_data.groupby("ip_os_device").cumcount()
click_data["rank_by_ip_os_device_dow"] = click_data.groupby("ip_os_device_dow").cumcount()

CPU times: user 2min 26s, sys: 25.8 s, total: 2min 51s
Wall time: 2min 51s


In [42]:
%%time
click_data[["rank_by_ip", "rank_by_ip_os_device", "rank_by_ip_os_device_dow"]].to_hdf("../data/ranks.hdf", "ranks")

CPU times: user 3.53 s, sys: 8.05 s, total: 11.6 s
Wall time: 11.6 s


In [3]:
time_ip = click_data[["click_time", "ip"]]

In [4]:
%%time
time_ip.sort_values(by=["ip", "click_time"], inplace=True)

CPU times: user 3min 8s, sys: 13.1 s, total: 3min 21s
Wall time: 3min 21s


/home/ubuntu/.pyenv/versions/3.6.4/envs/talkingdata/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [5]:
%%time
time_ip["prev_time"] = time_ip["click_time"].shift()
time_ip["prev_ip"] = time_ip["ip"].shift()

/home/ubuntu/.pyenv/versions/3.6.4/envs/talkingdata/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


CPU times: user 1.51 s, sys: 2.06 s, total: 3.58 s
Wall time: 3.58 s


/home/ubuntu/.pyenv/versions/3.6.4/envs/talkingdata/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [6]:
%%time
time_ip["click_time_interval_by_ip"] = time_ip["click_time"] - time_ip["prev_time"]

CPU times: user 59.2 s, sys: 3.6 s, total: 1min 2s
Wall time: 1min 1s


/home/ubuntu/.pyenv/versions/3.6.4/envs/talkingdata/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [7]:
time_ip["click_time_interval_by_ip"]=time_ip["click_time_interval_by_ip"].where(time_ip["ip"] == time_ip["prev_ip"], np.nan)

/home/ubuntu/.pyenv/versions/3.6.4/envs/talkingdata/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
%%time
time_ip.sort_index(inplace=True)

CPU times: user 47.8 s, sys: 7.02 s, total: 54.8 s
Wall time: 54.8 s


/home/ubuntu/.pyenv/versions/3.6.4/envs/talkingdata/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
%%time
time_ip[["click_time_interval_by_ip"]].to_hdf("../data/click_time_interval_by_ip.hdf", "click_time_interval_by_ip")

CPU times: user 1.18 s, sys: 3.15 s, total: 4.33 s
Wall time: 4.33 s


In [12]:
del time_ip
gc.collect()

260

In [21]:
%%time
time_ip_os_device = click_data[["click_time", "ip_os_device"]]
time_ip_os_device.sort_values(by=["ip_os_device", "click_time"], inplace=True)
time_ip_os_device["prev_time"] = time_ip_os_device["click_time"].shift()
time_ip_os_device["prev_ip_os_device"] = time_ip_os_device["ip_os_device"].shift()
time_ip_os_device["click_time_interval_by_ip_os_device"] = time_ip_os_device["click_time"] - time_ip_os_device["prev_time"]
time_ip_os_device["click_time_interval_by_ip_os_device"] = time_ip_os_device["click_time_interval_by_ip_os_device"].where(time_ip_os_device["ip_os_device"] == time_ip_os_device["prev_ip_os_device"], np.nan)
time_ip_os_device.sort_index(inplace=True)

/home/ubuntu/.pyenv/versions/3.6.4/envs/talkingdata/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ubuntu/.pyenv/versions/3.6.4/envs/talkingdata/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ubuntu/.pyenv/versions/3.6.4/envs/talkingdata/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

CPU times: user 10min 12s, sys: 30.6 s, total: 10min 43s
Wall time: 10min 40s


/home/ubuntu/.pyenv/versions/3.6.4/envs/talkingdata/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [23]:
%%time
time_ip_os_device[["click_time_interval_by_ip_os_device"]].to_hdf("../data/click_time_interval_by_ip_os_device.hdf", "click_time_interval_by_ip_os_device")

CPU times: user 1.13 s, sys: 3.21 s, total: 4.34 s
Wall time: 4.33 s


In [24]:
%%time
time_dow = click_data[["click_time", "ip_os_device_dow"]]
time_dow.sort_values(by=["ip_os_device_dow", "click_time"], inplace=True)
time_dow["prev_time"] = time_dow["click_time"].shift()
time_dow["prev_ip_os_device_dow"] = time_dow["ip_os_device_dow"].shift()
time_dow["click_time_interval_by_ip_os_device_dow"] = time_dow["click_time"] - time_dow["prev_time"]
time_dow["click_time_interval_by_ip_os_device_dow"] = time_dow["click_time_interval_by_ip_os_device_dow"].where(time_dow["ip_os_device_dow"] == time_dow["prev_ip_os_device_dow"], np.nan)
time_dow.sort_index(inplace=True)
time_dow[["click_time_interval_by_ip_os_device_dow"]].to_hdf("../data/click_time_interval_by_ip_os_device_dow.hdf", "click_time_interval_by_ip_os_device_dow")

/home/ubuntu/.pyenv/versions/3.6.4/envs/talkingdata/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ubuntu/.pyenv/versions/3.6.4/envs/talkingdata/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ubuntu/.pyenv/versions/3.6.4/envs/talkingdata/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

CPU times: user 5min 23s, sys: 34 s, total: 5min 57s
Wall time: 5min 55s


In [25]:
%%time
time_hour = click_data[["click_time", "ip_os_device_dow_hour"]]
time_hour.sort_values(by=["ip_os_device_dow_hour", "click_time"], inplace=True)
time_hour["prev_time"] = time_hour["click_time"].shift()
time_hour["prev_ip_os_device_dow_hour"] = time_hour["ip_os_device_dow_hour"].shift()
time_hour["click_time_interval_by_ip_os_device_dow_hour"] = time_hour["click_time"] - time_hour["prev_time"]
time_hour["click_time_interval_by_ip_os_device_dow_hour"] = time_hour["click_time_interval_by_ip_os_device_dow_hour"].where(time_hour["ip_os_device_dow_hour"] == time_hour["prev_ip_os_device_dow_hour"], np.nan)
time_hour.sort_index(inplace=True)
time_hour[["click_time_interval_by_ip_os_device_dow_hour"]].to_hdf("../data/click_time_interval_by_ip_os_device_dow_hour.hdf", "click_time_interval_by_ip_os_device_dow_hour")

/home/ubuntu/.pyenv/versions/3.6.4/envs/talkingdata/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ubuntu/.pyenv/versions/3.6.4/envs/talkingdata/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ubuntu/.pyenv/versions/3.6.4/envs/talkingdata/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

CPU times: user 5min 37s, sys: 57.5 s, total: 6min 34s
Wall time: 6min 32s
